# Toxikind: Data Exploration + Baseline Re

In [23]:
# OS I/O
import os
import pickle

# Data handling
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

# Modeling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate, GridSearchCV #, cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay #, classification_report# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

Pure Preprocessing:
1. Import Raw Data
2. Scaling Pipeline (Features)
3. Merge by Index (all the data) (export)
-> .py-file and scaled full dataset in data folder

Modeling: (assumption: NAs are removed)
1. Select Target(s) by user/environement list
2. Put in function which selects the target(s) of interest (by removing the other) and drops all NAs

# Features (Train)

**Load data:**

In [6]:
path_features_train = "../raw_data/tox21_dense_train.csv.gz"
X_raw = pd.read_csv(path_features_train).set_index("Unnamed: 0")

**Shape:**

In [7]:
X_raw.shape

(12060, 801)

**Table info:**

In [8]:
X_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12060 entries, NCGC00178831-03 to NCGC00261465-01
Columns: 801 entries, AW to rygr
dtypes: float64(761), int64(40)
memory usage: 73.8+ MB


**Table view:**

**Pipeline: Scale all features and retain colnames + index:**

In [9]:
# Column Transformer with MinMax-Scaler
column_transformer = ColumnTransformer([
    ("scaler", MinMaxScaler(), X_raw.columns)
    ])

# Pipeline
pipeline = Pipeline([
    ("transform", column_transformer)
    ])

# Transform data while preserving column names
X = pd.DataFrame(pipeline.fit_transform(X_raw), columns=X_raw.columns, index=X_raw.index)

# Targets (Train)

**Load data:**

In [10]:
path_targets_train = "../raw_data/tox21_labels_train.csv.gz"
y = pd.read_csv(path_targets_train).set_index("Unnamed: 0")

**Shape:**

In [11]:
y.shape

(12060, 12)

**Table info:**

In [12]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12060 entries, NCGC00178831-03 to NCGC00261465-01
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NR.AhR         8441 non-null   float64
 1   NR.AR          9654 non-null   float64
 2   NR.AR.LBD      8852 non-null   float64
 3   NR.Aromatase   7440 non-null   float64
 4   NR.ER          7962 non-null   float64
 5   NR.ER.LBD      9040 non-null   float64
 6   NR.PPAR.gamma  8451 non-null   float64
 7   SR.ARE         7401 non-null   float64
 8   SR.ATAD5       9363 non-null   float64
 9   SR.HSE         8417 non-null   float64
 10  SR.MMP         7558 non-null   float64
 11  SR.p53         8903 non-null   float64
dtypes: float64(12)
memory usage: 1.2+ MB


**Table view:**

**Number of missing values per target:**

In [13]:
y.isnull().sum().sort_values(ascending=False)

SR.ARE           4659
NR.Aromatase     4620
SR.MMP           4502
NR.ER            4098
SR.HSE           3643
NR.AhR           3619
NR.PPAR.gamma    3609
NR.AR.LBD        3208
SR.p53           3157
NR.ER.LBD        3020
SR.ATAD5         2697
NR.AR            2406
dtype: int64

**Labels and NAs per target:**

In [14]:
# Calculate number of toxic, non toxic and NA
experiments = y.columns
exp_stats = []

for i in experiments:
    n_toxic = sum(y[i] ==1)
    n_non_toxic = sum(y[i] ==0)
    n_nan = sum(y[i].isna())
    percent_toxic = n_toxic / (n_toxic + n_non_toxic) * 100
    percent_non_toxic = 100 - percent_toxic
    exp_stats.append([i, n_toxic, n_non_toxic, n_nan, percent_toxic, percent_non_toxic])

In [ ]:
# As DataFrame
stats_df = pd.DataFrame(exp_stats, columns=["experiment", "toxic", "non_toxic", "NA", "percent_toxic", "percent_non_toxic"])
stats_df

In [ ]:
# Plot toxic, non toxic, NA
stats_df[["experiment", "toxic", "non_toxic", "NA"]]\
    .set_index("experiment").plot(kind="bar", stacked=True, figsize=(12, 6))
plt.title("Distribution of Toxic, Non-Toxic, and Missing Labels per Experiment")
plt.ylabel("Count")
plt.xlabel("Experiment")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()

**Labels per target:**

In [ ]:
# Generate the stacked bar chart
ax = stats_df[["experiment", "percent_toxic", "percent_non_toxic"]].set_index("experiment") \
    .plot(kind="bar", stacked=True, figsize=(12, 6))

# Extract experiment names (to match labels correctly)
experiment_names = stats_df["experiment"].values

# Add labels (loop through bars)
for bars, col in zip(ax.containers, ["percent_toxic", "percent_non_toxic"]):
    for bar, experiment in zip(bars, experiment_names):
        height = bar.get_height()  # Get the height of each segment
        if height > 0:  # Avoid labeling zero-height bars
            ax.text(bar.get_x() + bar.get_width() / 2,
                    bar.get_y() + height / 2,
                    f"{stats_df.loc[stats_df['experiment'] == experiment, col].values[0]:.1f}%",
                    ha="center", va="center", color="black", fontsize=9)

plt.title("Share of Toxic Labels per Experiment")
plt.ylabel("Count")
plt.xlabel("Experiment")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()
# Thank you, Microsoft Copilot!

**Correlation between targets:**

In [ ]:
# Calculate Correlation Matrix
target_corr = y.corr()

# Create Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(target_corr,
           annot=True,  # Show numbers
           cmap="coolwarm",  # Blue-red color scheme
           center=0,  # Center the colormap at 0
           fmt=".2f" # Round to 2 decimal places
           )

plt.title("Correlation Between Targets")
plt.tight_layout()
plt.show()

# Baseline Model: Random Forest 

### Target: "SR.MMP"

**Create single target without NAs:**

In [28]:
y_mmp = y[["SR.MMP"]].dropna()

In [29]:
y_mmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7558 entries, NCGC00165865-01 to NCGC00261465-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SR.MMP  7558 non-null   float64
dtypes: float64(1)
memory usage: 118.1+ KB


**Inner merge with features:**

In [30]:
mmp_train = y_mmp.merge(X, how="inner", on="Unnamed: 0")

In [31]:
mmp_train.head()

,SR.MMP,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NCGC00165865-01,1.0,6.341000e-08,0.017366,0.812523,0.789928,0.273763,0.270577,0.119708,0.282093,0.256368,...,0.041851,0.032067,0.195228,0.987276,0.964649,0.085498,0.016266,0.032846,0.000227,0.323018
NCGC00095025-01,1.0,4.232804e-01,0.009179,0.776528,0.677698,0.213232,0.215202,0.053085,0.195447,0.160676,...,0.019685,0.032279,0.042972,0.996306,0.977962,0.097667,0.024509,0.038591,0.000160,0.233044
NCGC00255209-01,1.0,8.695657e-02,0.012498,0.717285,0.649496,0.169998,0.168857,0.022973,0.148787,0.113646,...,0.014453,0.029822,0.053196,0.996467,0.968684,0.099991,0.019386,0.035609,0.000114,0.264252
NCGC00260005-01,1.0,3.760000e-08,0.005746,0.809899,0.755683,0.190918,0.187776,0.047752,0.192760,0.187370,...,0.014743,0.018509,0.058717,0.996961,0.985609,0.081238,0.012272,0.031258,0.000129,0.220536
NCGC00258363-01,1.0,5.882360e-02,0.022841,0.705662,0.636547,0.256603,0.257801,0.048654,0.195549,0.162270,...,0.046014,0.117323,0.077582,0.991233,0.939187,0.269463,0.088882,0.090879,0.000196,0.371946


### Initialize Decision Tree

**Split again:**

In [32]:
X_mmp_train = mmp_train.drop(columns=["SR.MMP"])
print(X_mmp_train.shape)
y_mmp_train = mmp_train["SR.MMP"]
print(y_mmp_train.shape)


(7558, 801)
(7558,)


**Model fit and evaluation:**

In [33]:
# Model Instantiation and Fit
random_forest = RandomForestClassifier()
random_forest.fit(X_mmp_train, y_mmp_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [36]:
# Cross-validation
logit_model_mmp_cv = cross_validate(random_forest
                                    , X=X_mmp_train, y=y_mmp_train, cv=5,
                                scoring=["accuracy",
                                         "recall",
                                         "precision",
                                         "f1"
                                        ]
                               )

In [38]:
logit_model_mmp_cv

{'fit_time': array([11.03505254, 11.70938492, 12.1213994 , 12.44789934, 11.49828959]),
 'score_time': array([0.04613018, 0.04075193, 0.03988028, 0.0377543 , 0.03630304]),
 'test_accuracy': array([0.86375661, 0.89880952, 0.88888889, 0.90205162, 0.88749173]),
 'test_recall': array([0.37711864, 0.44491525, 0.44915254, 0.4279661 , 0.31779661]),
 'test_precision': array([0.60135135, 0.82677165, 0.73611111, 0.88596491, 0.89285714]),
 'test_f1': array([0.46354167, 0.5785124 , 0.55789474, 0.57714286, 0.46875   ])}

In [37]:
logit_mmp_scoring_metrics = pd.DataFrame((list(logit_model_mmp_cv.values()))[2:]).mean(axis=1)
keys = ["accuracy", "recall", "precision", "f1"]
logit_scoring_metrics = dict(zip(keys, list(logit_mmp_scoring_metrics)))
logit_scoring_metrics

{'accuracy': 0.8881996750461344,
 'recall': 0.40338983050847455,
 'precision': 0.7886112342287228,
 'f1': 0.5291683314691688}

**Model fit and evaluation:**

In [30]:
# DOES NOT WORK YET!
model_path = os.path.join("./models", f"logit_mmp_{timestamp}.h5")
model.save(model_path)

NameError: name 'timestamp' is not defined

## Target: SR.ARE